In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
from sklearn import datasets

## 1. Membuat dataset

In [ ]:
  n_pts = 100 # data point
  centers = [[-0.5, 0.5], [0.5, -0.5]]
  X, y = datasets.make_blobs(n_samples=n_pts, random_state=123, centers=centers, cluster_std=0.4)
  
  x_data = torch.Tensor(X)
  y_data = torch.Tensor(y.reshape(100, 1))
  print(y.shape)

In [ ]:
def scatter_plot():
  plt.scatter(X[y==0, 0], X[y==0, 1])
  plt.scatter(X[y==1, 0], X[y==1, 1])

In [ ]:
scatter_plot()

## 2. Membuat kelas model

In [ ]:
class Model(nn.Module):
    def __init__(self, input_size, output_size):
      super().__init__() 
      self.linear = nn.Linear(input_size, output_size) # 1 neuron, input_size 2, X dan Y, output 1 karena class 1
    def forward(self, x):
      pred = torch.sigmoid(self.linear(x)) # non-linearity pakai sigmoid
      return pred
    def predict(self, x):
      pred = self.forward(x)
      if pred >= 0.5:
        return 1
      else:
        return 0

In [ ]:
torch.manual_seed(2)
model = Model(2, 1)
print(list(model.parameters()))

In [ ]:
[w, b] = model.parameters()
w1, w2 = w.view(2)
def get_params():
  return (w1.item(), w2.item(), b[0].item())

In [ ]:
def plot_fit(title):
  plt.title = title
  w1, w2, b1 = get_params()
  x1 = np.array([-2, 2])
  x2 = (w1*x1 + b1)/(-w2)
  plt.plot(x1, x2, 'r')
  scatter_plot()
  plt.show()

In [ ]:
plot_fit('Initial Model')

## 3. Loss Function dan Optimizer

In [ ]:
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [ ]:
epochs = 1000
losses = []

for i in range(epochs):
  y_pred = model.forward(x_data) # forward
  loss = criterion(y_pred, y_data) # compute loss
  print("epoch:", i, "loss:", loss.item())
  losses.append(loss.item())
  optimizer.zero_grad() # turunan untuk mencari nilai minimum
  loss.backward() # backward
  optimizer.step() # iterasi dan update weight

In [ ]:
plt.plot(range(epochs), losses)
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.grid()

## 4. Plot dari Model terbaik yang didapatkan

In [ ]:
plot_fit("Trained Model")

In [ ]:
get_params() # y = w[x1, x2] + w0
# y = [1.4355945587158203, -1.4284124374389648][x1,x2] + (-0.06880919635295868)

## Menggunakan model diatas untuk memprediksi

In [ ]:
point1 = torch.Tensor([1.0, -1.0])
point2 = torch.Tensor([-1.0, 1.0])
plt.plot(point1.numpy()[0], point1.numpy()[1], 'ro') # titik merah
plt.plot(point2.numpy()[0], point2.numpy()[1], 'ko') # titik hitam
plot_fit("Trained Model")
print("Titik orange probabilitas positif = {}".format(model.forward(point1).item())) 
print("Titik biru probabilitas positif = {}".format(model.forward(point2).item())) 
print("Titik merah ada dikelas = {}".format(model.predict(point1))) 
print("Titik hitam ada di kelas = {}".format(model.predict(point2))) 